In [1]:
import numpy as np
import pandas as pd
import re
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score,StratifiedKFold,GridSearchCV
#for rafi to run

C:\Users\rafiz\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
work_train= {} 
work_test = {}


pre_process=["wind_tfidf_100.csv","full_tfidf_100.csv","w2v_300.csv"]
path="./window/checkpoints_databases/"
work_train["bio"] = pd.read_csv(path + "new_working_train_bio.csv")
work_test["bio"] = pd.read_csv(path + "new_working_test_bio.csv")
for f in pre_process:
    work_train[re.sub("\.csv","",f)] = pd.read_csv(path+"new_stem_working_train_"+f)
    work_test[re.sub("\.csv","",f)] = pd.read_csv(path+"new_stem_working_test_"+f)

In [3]:
#External features + genes tsvd
feat_train_1=pd.read_csv("./window/w_meta_features/meta_train_l1l2.csv") #Gene tsvd
feat_test_1=pd.read_csv("./window/w_meta_features/meta_test_l1l2.csv") 
feat_train_1=feat_train_1.drop("ID",axis=1)
feat_test_1=feat_test_1.drop("ID",axis=1)
feat_train_2=pd.read_csv("./window/w_meta_features/new_working_train_ext.csv")#External tsvd
feat_test_2=pd.read_csv("./window/w_meta_features/new_working_test_ext.csv")

feat_test_1

,tsvd_gene0,tsvd_gene1,tsvd_gene2,tsvd_gene3,tsvd_gene4,tsvd_gene5,tsvd_gene6,tsvd_gene7,tsvd_gene8,tsvd_gene9,tsvd_gene10,tsvd_gene11,tsvd_gene12,tsvd_gene13,tsvd_gene14
0,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00
1,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00
2,3.186143e-28,6.314855e-22,-5.368292e-21,-1.236785e-21,-1.320999e-21,-2.451154e-21,3.177645e-20,-2.309820e-17,9.261944e-17,2.093526e-17,-8.205174e-17,-1.236460e-15,-9.665182e-17,-1.338068e-16,-1.333793e-16
3,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00
4,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00
5,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00
6,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00
7,-1.163553e-20,-7.881358e-20,2.818547e-19,1.081453e-14,1.000000e+00,-9.820299e-16,6.458034e-17,-4.024035e-18,2.197626e-17,-6.312353e-18,1.195702e-16,-3.355734e-17,-4.266393e-17,-9.126059e-18,8.254506e-20
8,1.000000e+00,1.516092e-16,-6.738994e-18,-8.014623e-21,4.496371e-21,6.232738e-22,5.423008e-23,1.730322e-24,-7.497432e-25,1.215962e-25,-9.461723e-26,-1.420038e-25,1.457221e-26,1.862061e-27,-5.626021e-26
9,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.000000e+00,-0.000000e+00,0.000000e+00


In [4]:
train=pd.concat((work_train["bio"],work_train["wind_tfidf_100"],work_train["full_tfidf_100"],work_train["w2v_300"],feat_train_1,feat_train_2),axis=1)

In [5]:
test=pd.concat((work_test["bio"],work_test["wind_tfidf_100"],work_test["full_tfidf_100"],work_test["w2v_300"],feat_test_1,feat_test_2),axis=1)

In [6]:
new_train=pd.read_csv("window/checkpoints_databases/new_working_train.csv")
new_test=pd.read_csv("window/checkpoints_databases/new_working_test.csv")
y=new_train.Class - 1
ID_train=new_train.ID
ID_test=new_test.ID
del new_train, new_test


In [7]:
kf=StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

In [8]:
X_train=np.array(train)
X_test=np.array(test)

In [16]:
clf_xgb=XGBClassifier(max_depth=5, objective="multi:softprob",seed=26)
h=cross_val_score(clf_xgb,np.array(train),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
print("mean : "+str(h.mean()),
         "std : "+name+" "+str(h.std()))

NameError: name 'name' is not defined

In [9]:
clf_xgb=XGBClassifier(max_depth=3, objective="multi:softprob",seed=26)
param_test= {
    "max_depth" : [3,5,7],
    "min_child_weight" : [1,3,5],
    "subsample":[0.8,0.9,1],
    "colsample_bytree":[0.8,0.9,1]
}
gsearch=GridSearchCV(estimator=clf_xgb, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
gsearch.fit(np.array(train),y)
print(gsearch.grid_scores_)
print(gsearch.best_params_)
print(gsearch.best_score_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 81.7min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed: 201.1min finished


[mean: -0.90729, std: 0.02711, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.8}, mean: -0.90395, std: 0.02758, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.9}, mean: -0.91300, std: 0.02275, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 1}, mean: -0.90548, std: 0.02403, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 3, 'subsample': 0.8}, mean: -0.91066, std: 0.02650, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 3, 'subsample': 0.9}, mean: -0.90776, std: 0.02286, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 3, 'subsample': 1}, mean: -0.90969, std: 0.02305, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 0.8}, mean: -0.91056, std: 0.02438, params: {'colsample_bytree': 0.8, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 0.9}, mean: -0.91347, std: 0.0201

C:\Users\rafiz\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [12]:
clf_xgb=XGBClassifier(max_depth=3, objective="multi:softprob",seed=26)
param_test= {
    "n_estimators" : [100, 200, 300, 400],
    "max_depth" : [5],
    "min_child_weight" : [5],
    "subsample": [1] ,
    "colsample_bytree": [1]
}
gsearch=GridSearchCV(estimator=clf_xgb, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
gsearch.fit(np.array(train),y)
print(gsearch.grid_scores_)
print(gsearch.best_params_)
print(gsearch.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 27.0min finished


[mean: -0.90191, std: 0.03144, params: {'colsample_bytree': 1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 100, 'subsample': 1}, mean: -0.97421, std: 0.04466, params: {'colsample_bytree': 1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 200, 'subsample': 1}, mean: -1.02597, std: 0.04945, params: {'colsample_bytree': 1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 300, 'subsample': 1}, mean: -1.06173, std: 0.05354, params: {'colsample_bytree': 1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 400, 'subsample': 1}]
{'colsample_bytree': 1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 100, 'subsample': 1}
-0.901910484291


C:\Users\rafiz\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [16]:
def model_gen(X,X_test,y,classifier):
    kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    fold = 0
    y_test=0
    for train_index, test_index in kf.split(X, y):
        
        fold += 1

        X_train, X_valid    = X[train_index],   X[test_index]
        y_train, y_valid    = y[train_index],   y[test_index]

        print("Fold", fold, X_train.shape, X_valid.shape)

        clf=classifier
        clf.fit(X_train,y_train)
        p_test = clf.predict_proba(X_test)
        y_test += p_test/5
    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("new_backup_sub.csv",index=False)
    

In [17]:
clf = XGBClassifier(n_estimators= 100, max_depth = 5, colsample_bytree = 1, min_child_weight= 5, subsample=1, objective="multi:softprob",seed=26)
model_gen(X_train,X_test,y,clf)

Fold 1 (2337, 750) (588, 750)
Fold 2 (2338, 750) (587, 750)
Fold 3 (2340, 750) (585, 750)
Fold 4 (2342, 750) (583, 750)
Fold 5 (2343, 750) (582, 750)


In [14]:
subm

NameError: name 'subm' is not defined